In [109]:
from amplpy import AMPL
import pandas as pd

In [110]:
rideshares = pd.read_csv('./rideshare_tg22.csv')
airports = ['LAX', 'ONT', 'SNA', 'BUR']
car_types = ['UberX', 'UberXL']

In [111]:
ez_rideshares = rideshares#[rideshares['Departure Date'] == '5/11/24']
ez_rideshares.rename(columns={'Departure Time (Pacific Daylight Time)': 'Flight depart'}, inplace=True)

In [112]:
cost_data = {
    ('LAX', 'UberX'): 86, ('LAX', 'UberXL'): 125,
    ('ONT', 'UberX'): 42, ('ONT', 'UberXL'): 80,
    ('SNA', 'UberX'): 75, ('SNA', 'UberXL'): 130,
    ('BUR', 'UberX'): 105, ('BUR', 'UberXL'): 150
}

In [113]:
capacity_data = {
    'UberX': 4,
    'UberXL': 6
}

In [114]:
travel_time_data = {
    'LAX': 1.5,
    'ONT': 0.5,
    'SNA': 1.5,
    'BUR': 1.5
}

In [115]:
pricing_multiplier = [1.00, 1.02, 1.02, 1.03, 1.03, 0.99, 1.04, 1.03, 1.03, 1.08, 1.08, 1.14, 1.14, 1.19, 1.19, 1.20, 1.20, 1.30, 1.14, 1.10, 1.13, 1.12, 1.15, 1.17, 1.16, 1.17, 1.17, 1.20, 1.20, 1.19, 1.23, 1.22, 1.26, 1.24, 1.27, 1.25, 1.25, 1.18, 1.18, 1.19, 1.19, 1.17, 1.08, 1.12, 1.12, 1.07, 1.07, 1.00 ]

# Initialize time-indexed cost data with correct time keys
time_indexed_cost_data = {}

# Populate cost data for each time period
for (origin, service), base_cost in cost_data.items():
    for t, multiplier in enumerate(pricing_multiplier, start=1):
        # Convert time index to match P (e.g., 0.5, 1.0, 1.5, ...)
        time_key = t * 0.5
        time_indexed_cost_data[(origin, service, time_key)] = round(base_cost * multiplier, 2)

# Example output
print(time_indexed_cost_data)


{('LAX', 'UberX', 0.5): 86.0, ('LAX', 'UberX', 1.0): 87.72, ('LAX', 'UberX', 1.5): 87.72, ('LAX', 'UberX', 2.0): 88.58, ('LAX', 'UberX', 2.5): 88.58, ('LAX', 'UberX', 3.0): 85.14, ('LAX', 'UberX', 3.5): 89.44, ('LAX', 'UberX', 4.0): 88.58, ('LAX', 'UberX', 4.5): 88.58, ('LAX', 'UberX', 5.0): 92.88, ('LAX', 'UberX', 5.5): 92.88, ('LAX', 'UberX', 6.0): 98.04, ('LAX', 'UberX', 6.5): 98.04, ('LAX', 'UberX', 7.0): 102.34, ('LAX', 'UberX', 7.5): 102.34, ('LAX', 'UberX', 8.0): 103.2, ('LAX', 'UberX', 8.5): 103.2, ('LAX', 'UberX', 9.0): 111.8, ('LAX', 'UberX', 9.5): 98.04, ('LAX', 'UberX', 10.0): 94.6, ('LAX', 'UberX', 10.5): 97.18, ('LAX', 'UberX', 11.0): 96.32, ('LAX', 'UberX', 11.5): 98.9, ('LAX', 'UberX', 12.0): 100.62, ('LAX', 'UberX', 12.5): 99.76, ('LAX', 'UberX', 13.0): 100.62, ('LAX', 'UberX', 13.5): 100.62, ('LAX', 'UberX', 14.0): 103.2, ('LAX', 'UberX', 14.5): 103.2, ('LAX', 'UberX', 15.0): 102.34, ('LAX', 'UberX', 15.5): 105.78, ('LAX', 'UberX', 16.0): 104.92, ('LAX', 'UberX', 16.5

In [116]:
ez_rideshares['Flight depart'] = pd.to_datetime(ez_rideshares['Flight depart'], format='%H:%M')
ez_rideshares['Flight times'] = ez_rideshares['Flight depart'].dt.hour + 0.5 * (ez_rideshares['Flight depart'].dt.minute//30)

In [117]:
ampl = AMPL()
ampl.eval(r"""
    set A;
    set S;
    set P = 0.5..24 by 0.5;  # time of arrival to airport
    set C;

    param cost{A, C, P};
    param capacity{C};
    param travelTime{A};
    param flightTime{S, A};

    var numCars{A, P, C} >= 0 integer;  # x
    var leaving{A, S, P} binary;  # 1 if student s leaves for airport a at time p, else 0
    var departureTime{A, S};

    minimize objective: sum{a in A, p in P, c in C} numCars[a, p, c]*cost[a, c, p];

    subject to notTooLateConstraint{a in A, s in S, p in P}:
        p - leaving[a, s, p]*(flightTime[s, a] - 2 - travelTime[a]) <= 100*(1 - leaving[a, s, p]);

    subject to notTooEarlyConstraint{a in A, s in S, p in P}:
        leaving[a, s, p]*(flightTime[s, a] - 4 - travelTime[a]) - p <= 100*(1 - leaving[a, s, p]);

    subject to capacityConstraint{a in A, p in P}:
        sum{s in S} leaving[a, s, p] <= sum{c in C} numCars[a, p, c] * capacity[c];

    subject to onlyOneActiveY{s in S}:
        sum{a in A, p in P} leaving[a, s, p] = 1;

    subject to defineDepartureTime{a in A, s in S}:
        departureTime[a, s] = sum{p in P} leaving[a, s, p] * p;

""")

In [118]:
# reset index on ez_rideshares to start at 0
ez_rideshares.reset_index(drop=True, inplace=True)

In [119]:
# set flight time data equal to Flight Times form ez_rideshares
# format: {(ez_rideshares['Name'][i], 'ONT') : ez_rideshares['Flight times'][i]}

raw_flight_time_data = {}
for i in range(len(ez_rideshares)):
    raw_flight_time_data[(ez_rideshares['Name'][i], 'ONT')] = ez_rideshares['Flight times'][i]

In [120]:
flight_time_data = {}
for s in ez_rideshares['Name']:
    for j in airports:
        if (s, j) in raw_flight_time_data:
            flight_time_data[(s, j)] = raw_flight_time_data[(s, j)]
        else:
            flight_time_data[(s, j)] = 0

In [121]:
# read ez_rideshares into ampl2
ampl.set['A'] = airports
ampl.set['S'] = ez_rideshares['Name']
ampl.set['C'] = car_types

ampl.getParameter("cost").setValues(time_indexed_cost_data)
ampl.getParameter("capacity").setValues(capacity_data)
ampl.getParameter("travelTime").setValues(travel_time_data)
ampl.getParameter("flightTime").setValues(flight_time_data)


In [122]:
# run the model
ampl.solve(solver='gurobi')

Gurobi 11.0.3:Gurobi 11.0.3: optimal solution; objective 340.2
310 simplex iterations
1 branching node


In [123]:
# display the results
student_airport_times = {}
for student in ez_rideshares['Name']:
    for a in airports:
        x = ampl.getValue(f'departureTime["{a}", "{student}"]')
        if x != 0:
            student_airport_times[student] = (a, x)
student_airport_times

{'Nithya Yeluri': ('ONT', 2.5),
 'Nina Jobanputra': ('ONT', 9.5),
 'Aanya Pratapneni ': ('ONT', 9.5),
 'Will Sedo': ('ONT', 9.5),
 'Marcella Todd': ('ONT', 11),
 'Kaanthi Pandhigunta': ('ONT', 11),
 'Cevi Bainton': ('ONT', 11),
 'Ally Dye': ('ONT', 13.5),
 'Maya Maranto': ('ONT', 13.5),
 'Alec Vercruysse': ('ONT', 13.5),
 'Rebecca Preis': ('ONT', 16),
 'Susan Li': ('ONT', 16),
 'Tanvi Krishnan ': ('ONT', 18),
 'Jennifer Li': ('ONT', 16),
 'Allison Marten': ('ONT', 16),
 'Aldrin Feliciano': ('ONT', 18),
 'Julianna Schalkwyk': ('ONT', 21),
 'Charles Weismann': ('ONT', 2.5),
 'Katrina Nguyen': ('ONT', 2.5),
 'Bennet Matazzoni': ('ONT', 11),
 'Thaxter Shaw': ('ONT', 18),
 'Alicia Krasner': ('ONT', 9.5),
 'Laura Vairus': ('ONT', 13.5),
 'Ashley Kim': ('ONT', 18)}

In [124]:
ampl.getValue('objective')

340.2

In [125]:
for a in airports:
    for i in range(1, 49):
        for car in car_types:
            numCars = ampl.getValue(f"numCars['{a}', {i/2}, '{car}']")
            if numCars > 0:
                print(f'Time {i/2} has {numCars} {car} car(s) to {a}')

Time 2.5 has 1 UberX car(s) to ONT
Time 9.5 has 1 UberX car(s) to ONT
Time 11.0 has 1 UberX car(s) to ONT
Time 13.5 has 1 UberX car(s) to ONT
Time 16.0 has 1 UberX car(s) to ONT
Time 18.0 has 1 UberX car(s) to ONT
Time 21.0 has 1 UberX car(s) to ONT
